# EDA (columnas, distribucion de datos, etc.)

In [0]:
from pyspark.sql.functions import month

# Setup

Leemos el archivo usando la libreria de pyspark, le indicamos el separador y vemos las columnas que contiene el dataset.

In [ ]:
# Leemos librerias he importamos el archivo usando spark
file_location = "/FileStore/tables/Hurto_a_personas_3_csv.csv"

df = (spark.read.format("csv")
        .option("header", "true")       
        .option("inferSchema", "true")  
        .option("sep", ";")             
        .option("encoding", "UTF-8")    
        .load(file_location))

print(df.columns)      
display(df.limit(5))

NameError: name 'spark' is not defined

In [0]:
# Reemplazamos los espacios en blanco y convertimos a minusculas
df = df.toDF(*[c.strip().replace(" ", "_").lower() for c in df.columns])
df.createOrReplaceTempView("hurto")


In [0]:
# Cambiamos los nombres de las columnas para que sean más amigables
%sql
SELECT year(fecha_hecho)  AS anio,
       month(fecha_hecho) AS mes,
       COUNT(*)           AS casos
FROM hurto
GROUP BY anio, mes
ORDER BY anio, mes;

anio,mes,casos
2025,1,22999
2025,2,22200
2025,3,23844


In [0]:
# vemos la cantidad de casos por municipio (top 10)
%sql
SELECT municipio, COUNT(*) AS casos
FROM hurto
GROUP BY municipio
ORDER BY casos DESC
LIMIT 10;

municipio,casos
Bogotá D.C. (CT),30963
Cali (CT),4374
Medellín (CT),4262
Barranquilla (CT),2671
Cartagena (CT),1728
Pasto (CT),1419
Bucaramanga (CT),1306
Villavicencio (CT),1232
Soacha,1132
Santa Marta (CT),1056


In [0]:
# visualizamos la cantidad de casos por mes (Como el dataset es del 2025 tiene pocos meses)
%sql
-- Serie mensual 2024-2025
SELECT date_format(fecha_hecho,'yyyy-MM') AS mes,
       SUM(cantidad)                       AS casos
FROM hurto
GROUP BY mes
ORDER BY mes;

mes,casos
2025-01,22999
2025-02,22200
2025-03,23844


In [0]:
# vemos la cantidad de casos por departamento 
%sql
-- Por departamento
WITH tmp AS (
  SELECT year(fecha_hecho)  AS anio,
         departamento,
         COUNT(*)           AS casos
  FROM hurto
  GROUP BY anio, departamento
)
SELECT * FROM tmp
ORDER BY anio, casos DESC;

anio,departamento,casos
2025,CUNDINAMARCA,34391
2025,ANTIOQUIA,6340
2025,VALLE,5670
2025,ATLÁNTICO,3658
2025,SANTANDER,2175
2025,BOLÍVAR,2086
2025,NARIÑO,1929
2025,META,1556
2025,HUILA,1539
2025,TOLIMA,1323


In [0]:
# Creamos un pivot para ver la cantidad de casos por mes y por armas_medios
# (armas_medios es una columna que tiene el tipo de arma o medio usado en el hurto)
pivot = (df.withColumn("mes", month("fecha_hecho"))
           .groupBy("mes")
           .pivot("armas_medios")
           .count()
           .orderBy("mes"))

display(pivot)

mes,ARMA BLANCA / CORTOPUNZANTE,ARMA DE FUEGO,CONTUNDENTES,ESCOPOLAMINA,LLAVE MAESTRA,NO REPORTADO,SIN EMPLEO DE ARMAS
1,3459,3749,726,119,3,781,14162
2,3521,3200,599,138,null,777,13965
3,3773,3458,581,117,4,881,15030


# Modelo
Vamos a predecir con que arma se realizara un determinado robo

In [0]:
# Importamos las librerias necesarias para el modelo
from pyspark.sql.functions import year, month
from pyspark.ml import Pipeline
from pyspark.ml.feature import (
    StringIndexer, OneHotEncoder, VectorAssembler)
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import IndexToString

In [0]:
# Renombramos la columna "agrupa_edad_persona" a "grupo_edad"
# y creamos una lista de columnas categoricas
# (las columnas categoricas son las que tienen el tipo de dato string)
df = spark.table("hurto")       
df.printSchema()
df = df.withColumnRenamed("agrupa_edad_persona", "grupo_edad")
categoricas = ["departamento", "municipio", "genero", "grupo_edad"]

root
 |-- armas_medios: string (nullable = true)
 |-- departamento: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- fecha_hecho: date (nullable = true)
 |-- genero: string (nullable = true)
 |-- agrupa_edad_persona: string (nullable = true)
 |-- codigo_dane: integer (nullable = true)
 |-- cantidad: integer (nullable = true)



In [0]:
# Las columnas que vamos a usar para el modelo
target       = "armas_medios"
categoricas  = ["departamento","municipio","genero","grupo_edad"]
numericas    = ["cantidad","anio","mes"]

df_rf = (df.withColumn("anio",  year("fecha_hecho"))
           .withColumn("mes",   month("fecha_hecho")))

# agregamos indices a cada variable categorica 1,2,3... etc
indexers = [
    StringIndexer(inputCol=c,
                  outputCol=f"{c}_idx",
                  handleInvalid="keep")
    for c in categoricas
]

# Usamos onehotencoder para las columnas categoricas
encoders = [
    OneHotEncoder(inputCol=f"{c}_idx",
                  outputCol=f"{c}_vec")
    for c in categoricas
]

# Juntamos todo en 1 solo vector
assembler = VectorAssembler(
    inputCols=[f"{c}_vec" for c in categoricas] + numericas,
    outputCol="features"
)

label_idx = StringIndexer(inputCol=target,
                          outputCol="label",
                          handleInvalid="keep")

# Modelo
rf = RandomForestClassifier(
        labelCol="label",
        featuresCol="features",
        numTrees=80,
        maxDepth=12,
        seed=42
)

pipeline = Pipeline(stages=indexers + encoders +
                    [assembler, label_idx, rf])

In [0]:
print(df_rf.columns)

['armas_medios', 'departamento', 'municipio', 'fecha_hecho', 'genero', 'grupo_edad', 'codigo_dane', 'cantidad', 'anio', 'mes']


In [0]:
# Entrenamos el modelo y hacemos un split de los datos
# (80% para entrenamiento y 20% para test)
train, test = df_rf.randomSplit([0.8, 0.2], seed=42)
model = pipeline.fit(train)
pred  = model.transform(test)

evaluator = MulticlassClassificationEvaluator(
               labelCol="label",
               predictionCol="prediction",
               metricName="accuracy")

accuracy = evaluator.evaluate(pred)
print(f"Accuracy: {accuracy:.3%}")

Accuracy: 62.427%


In [0]:
# Aqui convertimos el label a su nombre original
# (en vez de 0,1,2... etc a "arma blanca", "arma de fuego", "sin armas")
# (esto es para que sea más entendible)
label_indexer = model.stages[-2]           
converter = IndexToString(
              inputCol="prediction",
              outputCol="pred_armas_medios",
              labels=label_indexer.labels)

pred_human = converter.transform(pred)

In [0]:
# Limpiamos los nombres de las columnas
cols = ["departamento","municipio","fecha_hecho",
        "genero","grupo_edad",
        "armas_medios",      
        "pred_armas_medios"]  

display(pred_human.select(*cols).limit(20))

departamento,municipio,fecha_hecho,genero,grupo_edad,armas_medios,pred_armas_medios
AMAZONAS,Leticia (CT),2025-03-29,MASCULINO,ADULTOS,ARMA BLANCA / CORTOPUNZANTE,SIN EMPLEO DE ARMAS
ANTIOQUIA,Apartadó,2025-01-11,MASCULINO,ADULTOS,ARMA BLANCA / CORTOPUNZANTE,SIN EMPLEO DE ARMAS
ANTIOQUIA,Apartadó,2025-03-10,MASCULINO,ADULTOS,ARMA BLANCA / CORTOPUNZANTE,SIN EMPLEO DE ARMAS
ANTIOQUIA,Bello,2025-01-02,MASCULINO,ADULTOS,ARMA BLANCA / CORTOPUNZANTE,SIN EMPLEO DE ARMAS
ANTIOQUIA,Bello,2025-01-11,MASCULINO,ADULTOS,ARMA BLANCA / CORTOPUNZANTE,SIN EMPLEO DE ARMAS
ANTIOQUIA,Bello,2025-01-18,MASCULINO,ADULTOS,ARMA BLANCA / CORTOPUNZANTE,SIN EMPLEO DE ARMAS
ANTIOQUIA,Bello,2025-02-06,MASCULINO,ADULTOS,ARMA BLANCA / CORTOPUNZANTE,SIN EMPLEO DE ARMAS
ANTIOQUIA,Bello,2025-02-16,MASCULINO,ADULTOS,ARMA BLANCA / CORTOPUNZANTE,SIN EMPLEO DE ARMAS
ANTIOQUIA,Bello,2025-02-24,MASCULINO,ADULTOS,ARMA BLANCA / CORTOPUNZANTE,SIN EMPLEO DE ARMAS
ANTIOQUIA,Bello,2025-02-26,FEMENINO,ADULTOS,ARMA BLANCA / CORTOPUNZANTE,SIN EMPLEO DE ARMAS


In [0]:
# Creamos una tabla para ver la cantidad de casos por mes y por armas_medios
(confusion := pred_human
    .groupBy("armas_medios","pred_armas_medios")
    .count()
    .orderBy("armas_medios","pred_armas_medios")
).show(truncate=False)

+---------------------------+-------------------+-----+
|armas_medios               |pred_armas_medios  |count|
+---------------------------+-------------------+-----+
|ARMA BLANCA / CORTOPUNZANTE|SIN EMPLEO DE ARMAS|2073 |
|ARMA DE FUEGO              |SIN EMPLEO DE ARMAS|2105 |
|CONTUNDENTES               |SIN EMPLEO DE ARMAS|390  |
|ESCOPOLAMINA               |SIN EMPLEO DE ARMAS|104  |
|LLAVE MAESTRA              |SIN EMPLEO DE ARMAS|2    |
|NO REPORTADO               |SIN EMPLEO DE ARMAS|504  |
|SIN EMPLEO DE ARMAS        |SIN EMPLEO DE ARMAS|8603 |
+---------------------------+-------------------+-----+

